In [1]:
%pylab inline
import sympy as sp
sp.init_printing()

Populating the interactive namespace from numpy and matplotlib


# Sadd, Exercise 1.1

For the given matrix/vector pairs, compute the following quantities:

- $a_{ii}$
- $a_{ij}a_{ij}$
- $a_{ij}a_{jk}$
- $a_{ij}b_{j}$
- $a_{ij}b_{i}b_{j}$
- $b_{i}b_{j}$
- $b_{i}b_{i}$

For each case, point out whether the result is a scalar, vector, or matrix.

a) $a_{ij} = \begin{bmatrix} 1 & 1 & 1 \\ 0 & 4 & 2 \\ 0 & 1 & 1 \end{bmatrix}$,
   $b_{i} = \begin{Bmatrix} 1 \\ 0 \\ 2 \end{Bmatrix}$
   
b) $a_{ij} = \begin{bmatrix} 1 & 2 & 0 \\ 0 & 2 & 1 \\ 0 & 4 & 2 \end{bmatrix}$,
   $b_{i} = \begin{Bmatrix} 2 \\ 1 \\ 1 \end{Bmatrix}$

c) $a_{ij} = \begin{bmatrix} 1 & 1 & 1 \\ 1 & 0 & 2 \\ 0 & 1 & 4 \end{bmatrix}$,
   $b_{i} = \begin{Bmatrix} 1 \\ 1 \\ 0 \end{Bmatrix}$

## Solution

In the following, I define functions for computing matrix operations.  Optimized versions of each one of these functions are available through numpy/scipy but I define them to show the explicit implementation the sums represented by the index notation.

In [2]:
a, b = {}, {}
a['a'], b['a'] = array([[1, 1, 1], [0, 4, 2], [0, 1, 1]]), array([1, 0, 2])
a['b'], b['b'] = (array([[1, 2, 0], [0, 2, 1], [0, 4, 2]]), array([2, 1, 1]))
a['c'], b['c'] = array([[1, 1, 1], [1, 0, 2], [0, 1, 4]]), array([1, 1, 0])

- $a_{ii} = \sum_{i=1}^n a_{ii} = a_{11} + a_{22} + \cdots + a_{nn} = tr(\boldsymbol{A})$ is a scalar

In [23]:
def tr(a):
    """Compute the trace of a matrix a"""
    x = 0
    for i in range(a.shape[0]):
        x += a[i,i]
    return x

In [24]:
a1 = tr(a['a'])
b1 = tr(a['b'])
c1 = tr(a['c'])
a1, b1, c1

(6, 5, 5)

- $
a_{ij}a_{ij} = \sum_{i=1}^m \sum_{j=1}^n = a_{11}a_{11} + a_{12}a_{12} + \cdots a_{mn} a_{mn} = \boldsymbol{A}:\boldsymbol{A}
$
is a scalar

In [25]:
def double_dot(a, b):
    """Compute the double dot product of a and b"""
    x = 0
    for i in range(a.shape[0]):
        for j in range(b.shape[1]):
            x += a[i,j] * b[i,j]
    return x

In [26]:
a2 = double_dot(a['a'], a['a'])
b2 = double_dot(a['b'], a['b'])
c2 = double_dot(a['c'], a['c'])
a2, b2, c2

(25, 30, 25)

- $a_{ij}a_{jk} = \sum_{j=1}^m a_{ij} a_{jk} = \boldsymbol{A}\cdot\boldsymbol{A}$ is a matrix

In [27]:
def vecdot(a, b):
    """Compute the dot product of two vectors"""
    x = 0
    for i in range(a.shape[0]):
        x += a[i] * b[i]
    return x

def matmul(a, b):
    """Compute x = a.b"""
    assert a.shape[1] == b.shape[0]
    x = zeros((a.shape[0], b.shape[1]))
    for i in range(a.shape[0]):
        for k in range(b.shape[1]):
            x[i,k] = vecdot(a[i,:], b[:,k])
    return x

In [28]:
a3 = matmul(a['a'], a['a'])
b3 = matmul(a['b'], a['b'])
c3 = matmul(a['c'], a['c'])
a3, b3, c3

(array([[  1.,   6.,   4.],
        [  0.,  18.,  10.],
        [  0.,   5.,   3.]]), array([[  1.,   6.,   2.],
        [  0.,   8.,   4.],
        [  0.,  16.,   8.]]), array([[  2.,   2.,   7.],
        [  1.,   3.,   9.],
        [  1.,   4.,  18.]]))

- $a_{ij}b_{j} = \sum_{j=1}^m a_{ij} b_j = \boldsymbol{A}\cdot\boldsymbol{b}$ is a vector

In [33]:
def matvec(a, b):
    """Compute the matrix/vector product x = a.b"""
    x = zeros(b.shape[0])
    for i in range(a.shape[1]):
        x[i] = vecdot(a[i,:], b)
    return x

In [34]:
a4 = matvec(a['a'], b['a'])
b4 = matvec(a['b'], b['b'])
c4 = matvec(a['c'], b['c'])
a4, b4, c4

(array([ 3.,  4.,  2.]), array([ 4.,  3.,  6.]), array([ 2.,  1.,  1.]))

- $a_{ij}b_{i}b_{j} = \sum_{i=1}^m \sum_{j=1}^n a_{11}b_1b_1 + a_{12} b_1 b_2 + \cdots + a_{mn} b_m b_n = \boldsymbol{A}\cdot\boldsymbol{b}\cdot\boldsymbol{b}$ is a scalar

In [35]:
a5 = vecdot(matvec(a['a'], b['a']), b['a'])
b5 = vecdot(matvec(a['b'], b['b']), b['b'])
c5 = vecdot(matvec(a['c'], b['c']), b['c'])
a5, b5, c5

- $b_{i}b_{j} = B_{ij}$ is a matrix with components $b_i b_j$

In [18]:
def dyad(a, b):
    """Compute the dyad x, with x_ij = a_i b_j"""
    x = zeros((a.shape[0], b.shape[0]))
    for i in range(a.shape[0]):
        for j in range(b.shape[0]):
            x[i,j] = a[i] * b[j]
    return x

In [19]:
a6 = dyad(b['a'], b['a'])
b6 = dyad(b['b'], b['b'])
c6 = dyad(b['c'], b['c'])
a6, b6, c6

(array([[ 1.,  0.,  2.],
        [ 0.,  0.,  0.],
        [ 2.,  0.,  4.]]), array([[ 4.,  2.,  2.],
        [ 2.,  1.,  1.],
        [ 2.,  1.,  1.]]), array([[ 1.,  1.,  0.],
        [ 1.,  1.,  0.],
        [ 0.,  0.,  0.]]))

- $b_{i}b_{i} = \sum_{i=1}^n b_i b_i = b_1 b_1 + b_2 b_2 + \cdots + b_n b_n = \boldsymbol{b}\cdot\boldsymbol{b}$ is a scalar

In [37]:
a7 = vecdot(b['a'], b['a'])
b7 = vecdot(b['b'], b['b'])
c7 = vecdot(b['c'], b['c'])
a7, b7, c7

(5, 6, 2)